<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/workshop/workshop_setup_2pub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set variables

In [ ]:
project = "workshop01"
sdr_name = "sdr-04"
adv_name = "adv-08"
worker_name = "worker-01"

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

openrc_path = "/content/" + project + "-openrc.sh"

with open(openrc_path, 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools, get_available_publicips, get_segment_ids, get_radio_interfaces, get_worker_interfaces

# Reserve resources

In the next cells, we reserve 1 SDR, 1 worker, and 1 Advantech 5G router.

In [ ]:
# Check the SDR's health and the status of its ports, both ports are supposed to be up, otherwise contact support
sdr_status = get_radio_interfaces(sdr_name)
logger.info(f"{json.dumps(sdr_status, indent=4)}")
for port in sdr_status.keys():
  if sdr_status[port]['linkstate'] == 'Down':
    logger.warning(f"port {port} on {sdr_name} is down.")
  if sdr_status[port]['linkstate'] == 'Up':
    logger.success(f"port {port} on {sdr_name} is up.")

In [ ]:
# Reserve the SDR
segment_ids = get_segment_ids(sdr_name)

# reserve RJ45 port
rj45_lease = show_reservation_byname(sdr_name + "-rj45-lease")
if not rj45_lease:
    rj45_lease = reserve(
        { "type":"network", "name": sdr_name+"-rj45", "net_name": sdr_name+"-rj45", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# reserve SFP port
sfp_lease = show_reservation_byname(sdr_name + "-sfp-lease")
if not sfp_lease:
    sfp_lease = reserve(
        { "type":"network", "name": sdr_name+"-sfp", "net_name": sdr_name+"-sfp", "segment_id": segment_ids['sfp'], "duration": { "days":7, "hours":0 } }
    )

# reserve worker
worker_lease = show_reservation_byname(worker_name+"-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# reserve advantech router
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

# Run containers

Run a container for the advantech router

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
if len(available_pub_ips) == 0:
  raise Exception("There is no available public IPs to reserve.")
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
#     if interfaces[interface]['speed'] == '1000':
    available_ifs.append(interface)
# logger.info(f"Available 1Gbps interfaces on {worker_name}: {available_ifs}")
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
if len(available_ifs) < 2:
  logger.info(f"{json.dumps(interfaces, indent=4)}")
#   raise Exception(f"Did not find enough 1Gbps interfaces on {worker_name}")
  raise Exception(f"Did not find enough interfaces on {worker_name}")

# run the container
publicnet = chi.network.get_network("serverpublic")
advnet = chi.network.get_network(adv_name+"-net")
container_name = adv_name + "-end-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-dind",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },    nets = [
        { "network" : publicnet['id'] },
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[1],
        "networks.1.ip":pub_ip+"/27",
        "capabilities.privileged":"true",
        "networks.2.interface":available_ifs[0],
        "networks.2.ip":"10.42.3.2/24",
        "networks.2.routes":"192.168.70.128/26-10.42.3.1",

    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

# install iperf3 + ip utilities in the container
command = "apt-get update && apt-get install -y iproute2 iperf3 iputils-ping tshark tcpdump"
result = chi.container.execute(
    container_ref=container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

Run a container for the SDR, with a public IP

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
if len(available_pub_ips) == 0:
  raise Exception("There is no available public IPs to reserve.")
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces on the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
# we need one 10Gbps interface for the SDR,
# and one port with any speed for the public interface
port_10g = None
port_any = None
for interface in available_ifs:
  if (port_10g is None) and (interfaces[interface]['speed'] == '10000'):
    port_10g = interface
    continue
  if port_any is None:
    port_any = interface
if (port_10g is None) or (port_any is None):
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find proper interfaces on {worker_name}")
else:
  logger.success(f"we choose {port_10g} for the SDR and {port_any} for the public access")

# run the container
sdrsfpnet = chi.network.get_network(sdr_name+"-sfp-net")
publicnet = chi.network.get_network("serverpublic")
container_name = sdr_name + "-public-host"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-dind-oai",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : sdrsfpnet['id'] },
    ],
    labels = {
        "networks.1.interface":port_any,
        "networks.1.ip":pub_ip+"/27",
        "capabilities.privileged":"true",
        "networks.2.interface":port_10g,
        "networks.2.ip":"10.30.10.120/24",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")

# install iperf3 + ip utilities in the container
command = "apt-get update && apt-get install -y iproute2 iperf3 iputils-ping tshark tcpdump"
result = chi.container.execute(
    container_ref=container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")


# Connection verification

This cell can be run to test SDR connection from the advantech router end node, using iperf3 client. "iperf3 -s" server command is first needed on the SDR side.
The printout from iperf3 can then be viewed via the log window for the container.

In [ ]:
# Use iperf3 client to test the SDR connection
container_name = container_name = adv_name + "-end-node"
command = "iperf3 -c 192.168.70.129 -u -b 100M --get-server-output"
result = chi.container.execute(
    container_ref=container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")
